강화학습을 구성하는 여러 요소 중 3가지는 아래와 같다.

1. 액션 의존성- Agent가 하는 action에 따라 받는 reward가 다르다.

2. 시간 의존성- action에 대한 reward를 바로바로 받는 것이 아니다. 얼마의 시간이 경과한 뒤 reward를 받을 수도 있고 sparse하다.

3. 상태 의존성- Environment의 상태에 따라 받는 reward가 다르다.

이번 예시는 bandit을 당기면 그에 따라 다른 확률로 보상이나 패널티를 받는다. **멀티암드 밴딧**문제로서, 2, 3번을 고려하지 않는다.

아래는 강화학습 과정을 나타낸다.

1. Agent는 Environment로부터 받은 상태를 기반으로 action을 생산한다. (단, 이번 Bandit 예제에서는 Environement의 상태는 없다.)

2. Environment는 Agent로부터 받은 action을 기반으로 reward를 생산한다. 그리고 자신을 그 action을 기반으로 상태를 업데이트 한다. (단, Environment의 상태가 없기 때문에 생략한다.)

3. Agent는 앞서 생산했던 action을 주고 어떤 reward를 받았는지를 통해 학습을 진행한다.

loss는 -reward * log(weight)의 형태로 구하였는데, cross entropy를 취하는 것 같다. pytorch 내부 함수를 통해 구현을 대체하려고 하였으나 마땅한 함수가 없다.
* 살펴본 함수는 torch.nn.NLLLoss, torch.nn.CrossEntropyLoss로 얻을 수 있는 함수이다.
* 두 함수 모두 prediction, y를 입력으로 받는다.
* CrossEntropyLoss는 NLLLoss와 달리 y의 입력으로 [0, num_class)범위에 있는 수의 배열 뿐만 아니라 [0, 1]범위에 있는 num_class길이의 배열도 입력으로 받는다.
  * 따라서 action을 torch.nn.functional.one_hot으로 원핫 벡터로 바꾼 뒤 reward를 곱한 벡터를 y로 넣어주어 비슷하게 재현할 수 있다.
  * 하지만 입력으로 들어온 prediction에 log softmax를 적용하기 때문에 실제 구현과 달라진다.
* NLLLoss는 CrossEntropyLoss와 달리 prediction에 log softmax를 적용하지 않기 때문에 비슷하게 재현할 수 있다.
  * 하지만 CrossEntropyLoss에서처럼 [0, 1]범위에 있는 num_class길이의 배열도 y로 입력받을 수가 없다.

In [287]:
import numpy as np

class Environment:
    def __init__(self, bandit_arms):
        self.bandit_arms = list(bandit_arms)
        self.num_arms = len(bandit_arms)

    def pull_bandit(self, action):
        result = np.random.randn()
        return 1 if result > self.bandit_arms[action] else -1

env = Environment([0.2, 0, -0.2, -2])

In [288]:
import torch

pred = torch.tensor([1, 0.4, 0.1]).float()
y = torch.tensor(0).long()

a = torch.nn.CrossEntropyLoss(reduction='none')
b = torch.nn.LogSoftmax(dim=-1)
c = torch.nn.NLLLoss(reduction='none')

print(a(pred, y))
print(b(pred))
print(c(b(pred), y))

tensor(0.6706)
tensor([-0.6706, -1.2706, -1.5706])
tensor(0.6706)


In [289]:
import torch

class Agent():
    def __init__(self, num_actions):
        self.weights = torch.ones(num_actions, requires_grad=True)
        self.policy = torch.nn.Softmax(dim=0)
        self.optimizer = torch.optim.Adam([self.weights], lr=1e-3)

    def action(self) -> int:
        with torch.inference_mode():
            output = self.policy(self.weights)
            action = torch.multinomial(output, 1)
        return action.detach().item()

    def train(self, action, reward):
        responsible_output = self.weights[action]
        loss = -reward * torch.log(responsible_output)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

agent = Agent(env.num_arms)

In [290]:
total_episode = 1000
total_reward = np.zeros(env.num_arms)

np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

for i in range(total_episode):
    action = agent.action()
    reward = env.pull_bandit(action)
    agent.train(action, reward)
    total_reward[action] += reward
    if i % 50 == 49:
        print(i + 1, total_reward)

50 [-4. -2. -3. 11.]
100 [-2. -4. -3. 23.]
150 [-6. -7.  1. 34.]
200 [-7. -6.  7. 46.]
250 [ 0. -6. 10. 58.]
300 [ 0. -2. 12. 76.]
350 [ 1. -2. 12. 93.]
400 [  0.   1.  15. 110.]
450 [  3.  -3.  20. 126.]
500 [ -6.  -5.  26. 141.]
550 [-15.  -4.  28. 153.]
600 [-18.  -5.  27. 170.]
650 [-21.  -3.  28. 184.]
700 [-26.  -4.  27. 197.]
750 [-29.  -4.  29. 214.]
800 [-29.   0.  26. 229.]
850 [-32.   2.  25. 245.]
900 [-29.   5.  31. 255.]
950 [-28.   6.  36. 272.]
1000 [-29.   1.  41. 281.]


In [291]:
agent.weights

tensor([0.9417, 1.0032, 1.0806, 1.4921], requires_grad=True)